In [1]:
import sys
sys.path.append('../')
import pandas as pd
df = pd.DataFrame([
  {"question": "In quantum mechanics, what principle asserts that it's impossible to simultaneously know the exact position and momentum of a particle?", "expected_answer": "Heisenberg Uncertainty Principle"},
  {"question": "Which famous poet wrote 'The Love Song of J. Alfred Prufrock'?", "expected_answer": "T.S. Eliot"},
  {"question": "What mathematical theorem states that in any right-angled triangle, the area of the square whose side is the hypotenuse is equal to the sum of the areas of the squares whose sides are the two legs?", "expected_answer": "Pythagorean Theorem"},
  {"question": "Which philosophical paradox involves a ship where all of its wooden parts are replaced with metal parts?", "expected_answer": "Ship of Theseus"},
])
df

,question,expected_answer
0,"In quantum mechanics, what principle asserts t...",Heisenberg Uncertainty Principle
1,Which famous poet wrote 'The Love Song of J. A...,T.S. Eliot
2,What mathematical theorem states that in any r...,Pythagorean Theorem
3,Which philosophical paradox involves a ship wh...,Ship of Theseus


In [2]:
from ally.skills.generation.qa import QuestionAnsweringSkill

qa_skill = QuestionAnsweringSkill(
  input_template="Question: {question}",
  name='qa_skill',
)

In [3]:
from ally.runtimes.openai import OpenAIRuntime
from app.core.settings import settings

openai_runtime = OpenAIRuntime(
    verbose=True,
    api_key=settings.openai_api_key,
    gpt_model_name="gpt-3.5-turbo",
    max_tokens=256,
  )

In [4]:
from ally.agents.base import Agent
from ally.environments.base import BasicEnvironment


agent = Agent(
	skills=qa_skill,
  runtimes={
    'openai': openai_runtime
	},
  teacher_runtimes={
    'openai': openai_runtime
	},
	environment=BasicEnvironment(
			ground_truth_dataset=df,
			ground_truth_columns={'qa_skill': 'expected_answer'}
	)
)

In [5]:
runtime = agent.get_runtime()
teacher_runtime = agent.get_teacher_runtime()

dataset = agent.environment.as_dataset()

# Apply agent skills to dataset and get experience with predictions
predictions = agent.skills.apply(dataset, runtime=runtime)

Applying skill: qa_skill

['question']

  0%|          | 0/4 [00:00<?, ?it/s]

{'question': "In quantum mechanics, what principle asserts that it's impossible to simultaneously know the exact 
position and momentum of a particle?", 'expected_answer': 'Heisenberg Uncertainty Principle'}

 50%|█████     | 2/4 [00:07<00:07,  3.75s/it]

...

 75%|███████▌  | 3/4 [00:11<00:03,  3.88s/it]

{'question': 'What mathematical theorem states that in any right-angled triangle, the area of the square whose side
is the hypotenuse is equal to the sum of the areas of the squares whose sides are the two legs?', 
'expected_answer': 'Pythagorean Theorem'}

100%|██████████| 4/4 [00:19<00:00,  5.34s/it]

{'question': 'Which philosophical paradox involves a ship where all of its wooden parts are replaced with metal 
parts?', 'expected_answer': 'Ship of Theseus'}

100%|██████████| 4/4 [00:42<00:00, 10.55s/it]


In [6]:
predictions

,question,expected_answer,qa_skill
0,"In quantum mechanics, what principle asserts t...",Heisenberg Uncertainty Principle,The Heisenberg uncertainty principle asserts t...
1,Which famous poet wrote 'The Love Song of J. A...,T.S. Eliot,T.S. Eliot
2,What mathematical theorem states that in any r...,Pythagorean Theorem,The mathematical theorem that states this is c...
3,Which philosophical paradox involves a ship wh...,Ship of Theseus,The philosophical paradox you are referring to...


In [12]:
from ally.utils.logs import print_text
agent.skills

LinearSkillSet(skills={'qa_skill': QuestionAnsweringSkill(name='qa_skill', instruction_template='Answer the question.', description='', input_template='Question: {question}', output_template=[{'name': 'answer', 'description': 'the the answer to the question'}], prediction_field='answer')}, skill_sequence=['qa_skill'])